# Pseudo Code
##  1. Data를 불러온다
##  2. r, 람다, 학습횟수를 정한다.
##  3. w 값을 초기화 한다.
##  4. Gradientdescent 함수를 구한다.
###     - Classification이 참인 경우
###     - Classification이 거짓인 경우
##   5. 루프를 반복하면서 w값을 업데이트 한다.
##   6. 결과를 출력한다.

In [6]:
import struct
import numpy as np
from sklearn.utils import shuffle
from sklearn import svm, metrics
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)

In [8]:
## loading mnist dataset

raw_train = read_idx("./data/train-patterns-idx3-ubyte")
train_data = np.reshape(raw_train, (60000, 28*28))
train_label = read_idx("./data/train-labels-idx1-ubyte")

raw_test = read_idx("./data/test-patterns-idx3-ubyte")
test_data = np.reshape(raw_test, (10000, 28*28))
test_label = read_idx("./data/test-labels-idx1-ubyte")

c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """


In [49]:
print(train_data.shape, train_label.shape)
print(test_data.shape, test_label.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [9]:
# Preprocessing
X_train_p = train_data/255
X_test_p = test_data/255

In [166]:
class SVM(object):
    def __init__(self, C=0.1, eta=0.001, batch_size=60, epochs=100, shuffle=True):
        self.C = C
        self.eta = eta
        self.batch_size = batch_size
        self.epochs = epochs
        self.shuffle = shuffle
        self.class_num = 0
    
    def fit(self, X, y, params=None, eval_score=None):
        xn, xf = np.shape(X) # xn: data points of X, xf: number of feature
        self.class_num = len(np.unique(y)) # 클래스 수 = 10
        
        y_ovr = self.encode_ovr(X, y)
        
        ## Initialize params: w, b
        if params is None:
            self.params = {
                'w_': np.random.randn(xf, self.class_num), #(784,10) 정규분포난수
                'b_': np.random.randn(1, self.class_num),
                'aver_w': np.random.randn(xf, self.class_num),
                'aver_b': np.random.randn(1, self.class_num)
            }
            w = self.params['w_']
            b = self.params['b_']
        
        if eval_score is None:
            self.score_val = 0
            
        cnt = 0 # SVM-SGD : k
        
        for epoch in range(0, self.epochs):
            if self.shuffle:
                X_shuffled, y_shuffled = self.shuffles(X, y_ovr)
            
            batch_count = int(xn/self.batch_size)
                
            for t in range(0, batch_count):
                X_batch, y_batch, bs = self.mini_batch(X_shuffled, y_shuffled, t)
                
                X_batch = np.reshape(X_batch, (bs, xf))
                y_batch = np.reshape(y_batch, (bs, self.class_num))
                
                loss = self.hinge_loss(X_batch, y_batch) # y*(X*W+b)
                
                loss = 1 - loss
                loss[loss <= 0] = 0
                loss[loss > 0] = 1
                # loss = np.mean(loss)
                
                dw = np.zeros(self.params['w_'].shape)
                db = np.zeros(self.params['b_'].shape)
                
                #fx = np.reshape(fx, (bs, self.class_num))
                #tmp = np.multiply(y, fx)
                #tmp = 1 - tmp
                
                #tmp[tmp <= 0] = 0
                #tmp[tmp > 0] = 1
                # y_tmp = np.multiply(y, loss.reshape(bs, self.class_num))
                y_tmp = loss * y_batch
                
                dw = -(1/bs) * np.dot(np.transpose(y_tmp), X_batch) + np.transpose((1/self.C) * self.params['w_'])
                db = -(1/bs) * np.sum(y_tmp, axis=0)
                
                self.params['w_'] = self.params['w_'] - np.transpose(self.eta * dw)
                self.params['b_'] = self.params['b_'] - self.eta * db
                
                cnt += 1
            
            temp_w = cnt * (cnt/(cnt+1))*self.params['aver_w'] + (1/(cnt+1))*self.params['w_']
            temp_b = cnt * (cnt/(cnt+1))*self.params['aver_b'] + (1/(cnt+1))*self.params['b_']
            
            prev_score = self.score_val
            pres_score = self.score(X, y)
            print("epochs: ", epoch)
            print("prev_score: %d", prev_score)
            print("pres_score: %d", pres_score)
            print('\n')
            if prev_score < pres_score:
                self.score_val = pres_score
            
            if self.det_weight(X, y, self.params['aver_w'], self.params['aver_b']) < self.det_weight(X, y, temp_w, temp_b):
                self.params['aver_w'] = temp_w
                self.params['aver_b'] = temp_b
        
        return self.params

    
    def shuffles(self, X, y):
        # Shuffle training data
        ran = np.arange(0, np.shape(X)[0])
        np.random.shuffle(ran)
        return X[ran], y[ran]
    
    
    def encode_ovr(self, X, y):
        # 1로 이루어진 배열(np.shape(y)[0], class_num)
        encode = np.ones((np.shape(y)[0], self.class_num))
        # one-hot encode     
        for i in range(self.class_num):
            encode[:, i][y != i] = -1
        return encode
    
    def mini_batch(self, X, y, t):
        xn = np.shape(X)[0]
        X_batch = X[t*self.batch_size : min(xn, (t+1)*self.batch_size)]
        y_batch = y[t*self.batch_size : min(xn, (t+1)*self.batch_size)]
        bs = min(xn, (t+1)*self.batch_size) - t*self.batch_size
        
        return X_batch, y_batch, bs
    
    def hinge_loss(self, X, y):
        fx = np.dot(X, self.params['w_']) + self.params['b_'] # X*W+b
        cfx = y * fx
        
        return cfx
    
    def predict(self, X):
        cla_score = np.dot(X, self.params['w_']) + self.params['b_']
        pred = np.argmax(cla_score, axis=1)
        
        return pred
    
    def score(self, X, y):
        pred = self.predict(X)
        score = np.mean(pred == y)
        
        return score
    
    def det_weight(self, X, y, w1, b1):
        temp = np.dot(X, w1) + b1
        temp = temp.T
        pred = np.argmax(temp, axis=1)
        sco = np.mean(pred == y)
        
        return sco
    
    def get_parameters(self):
        return self.params
    
    def get_params(self, deep=True):
        return {'C':self.C, 'batch_size':self.batch_size, 'epochs':self.epochs}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

In [142]:
X = X_train_p
y = train_label

In [143]:
svm = SVM()
svm.fit(X,y)

epochs:  0
prev_score: %d 0
pres_score: %d 0.12091666666666667




c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:135: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


epochs:  1
prev_score: %d 0.12091666666666667
pres_score: %d 0.1568


epochs:  2
prev_score: %d 0.1568
pres_score: %d 0.22888333333333333


epochs:  3
prev_score: %d 0.22888333333333333
pres_score: %d 0.2852


epochs:  4
prev_score: %d 0.2852
pres_score: %d 0.36538333333333334


epochs:  5
prev_score: %d 0.36538333333333334
pres_score: %d 0.4365833333333333


epochs:  6
prev_score: %d 0.4365833333333333
pres_score: %d 0.5406166666666666


epochs:  7
prev_score: %d 0.5406166666666666
pres_score: %d 0.6167166666666667


epochs:  8
prev_score: %d 0.6167166666666667
pres_score: %d 0.7029666666666666


epochs:  9
prev_score: %d 0.7029666666666666
pres_score: %d 0.7375


epochs:  10
prev_score: %d 0.7375
pres_score: %d 0.7395666666666667


epochs:  11
prev_score: %d 0.7395666666666667
pres_score: %d 0.7031833333333334


epochs:  12
prev_score: %d 0.7395666666666667
pres_score: %d 0.69265


epochs:  13
prev_score: %d 0.7395666666666667
pres_score: %d 0.7255833333333334


epochs:  14
prev_scor

{'w_': array([[ 2.42092166e-322,  2.42092166e-322, -2.42092166e-322, ...,
          2.42092166e-322,  2.42092166e-322,  2.42092166e-322],
        [ 2.42092166e-322,  2.42092166e-322, -2.42092166e-322, ...,
         -2.42092166e-322, -2.42092166e-322, -2.42092166e-322],
        [-2.42092166e-322, -1.58610303e-009, -2.42092166e-322, ...,
         -2.42092166e-322, -2.42092166e-322, -2.42092166e-322],
        ...,
        [-6.68971254e-008, -2.42092166e-322, -1.81921272e-012, ...,
         -7.08424128e-012, -2.42092166e-322,  4.06130608e-007],
        [ 2.42092166e-322, -2.42092166e-322,  2.42092166e-322, ...,
         -2.42092166e-322, -2.42092166e-322, -2.42092166e-322],
        [-2.42092166e-322, -2.42092166e-322,  2.42092166e-322, ...,
          2.42092166e-322, -2.42092166e-322,  2.42092166e-322]]),
 'b_': array([[-1.02763591, -0.96849863, -1.0118992 , -1.01057124, -1.00079266,
         -0.99904366, -1.01029002, -0.99531236, -1.01586011, -1.00476595]]),
 'aver_w': array([[ 0.51789489

In [130]:
scr = svm.score(X_test_p, test_label)
scr

0.6922

In [161]:
# GridSearchCV with Customized model SVM
from sklearn.model_selection import GridSearchCV
params = {
    'C':[100, 1000, 3000],
#     'C':[0.01, 0.1, 1, 10, 100, 1000, 3000]
     'batch_size':[60, 100, 600],
     'epochs':[50, 200]
}

gs = GridSearchCV(SVM(), param_grid=params, cv=3)

In [151]:
cv = gs.fit(X_train_p, train_label)

epochs:  0
prev_score: %d 0
pres_score: %d 0.123525




c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:135: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


epochs:  1
prev_score: %d 0.123525
pres_score: %d 0.1929


epochs:  2
prev_score: %d 0.1929
pres_score: %d 0.32805


epochs:  3
prev_score: %d 0.32805
pres_score: %d 0.485075


epochs:  4
prev_score: %d 0.485075
pres_score: %d 0.58125


epochs:  5
prev_score: %d 0.58125
pres_score: %d 0.6126


epochs:  6
prev_score: %d 0.6126
pres_score: %d 0.626725


epochs:  7
prev_score: %d 0.626725
pres_score: %d 0.62835


epochs:  8
prev_score: %d 0.62835
pres_score: %d 0.635425


epochs:  9
prev_score: %d 0.635425
pres_score: %d 0.6373


epochs:  10
prev_score: %d 0.6373
pres_score: %d 0.639525


epochs:  11
prev_score: %d 0.639525
pres_score: %d 0.653575


epochs:  12
prev_score: %d 0.653575
pres_score: %d 0.653725


epochs:  13
prev_score: %d 0.653725
pres_score: %d 0.65895


epochs:  14
prev_score: %d 0.65895
pres_score: %d 0.662675


epochs:  15
prev_score: %d 0.662675
pres_score: %d 0.666875


epochs:  16
prev_score: %d 0.666875
pres_score: %d 0.672


epochs:  17
prev_score: %d 0.672
pres_sc

epochs:  37
prev_score: %d 0.70305
pres_score: %d 0.69375


epochs:  38
prev_score: %d 0.70305
pres_score: %d 0.692


epochs:  39
prev_score: %d 0.70305
pres_score: %d 0.69485


epochs:  40
prev_score: %d 0.70305
pres_score: %d 0.6958


epochs:  41
prev_score: %d 0.70305
pres_score: %d 0.69965


epochs:  42
prev_score: %d 0.70305
pres_score: %d 0.6938


epochs:  43
prev_score: %d 0.70305
pres_score: %d 0.686925


epochs:  44
prev_score: %d 0.70305
pres_score: %d 0.68735


epochs:  45
prev_score: %d 0.70305
pres_score: %d 0.702775


epochs:  46
prev_score: %d 0.70305
pres_score: %d 0.70115


epochs:  47
prev_score: %d 0.70305
pres_score: %d 0.70265


epochs:  48
prev_score: %d 0.70305
pres_score: %d 0.697875


epochs:  49
prev_score: %d 0.70305
pres_score: %d 0.706


epochs:  0
prev_score: %d 0
pres_score: %d 0.1222


epochs:  1
prev_score: %d 0.1222
pres_score: %d 0.1936


epochs:  2
prev_score: %d 0.1936
pres_score: %d 0.33125


epochs:  3
prev_score: %d 0.33125
pres_score: %d 0.48587

epochs:  123
prev_score: %d 0.7267
pres_score: %d 0.71305


epochs:  124
prev_score: %d 0.7267
pres_score: %d 0.71825


epochs:  125
prev_score: %d 0.7267
pres_score: %d 0.70845


epochs:  126
prev_score: %d 0.7267
pres_score: %d 0.71785


epochs:  127
prev_score: %d 0.7267
pres_score: %d 0.711525


epochs:  128
prev_score: %d 0.7267
pres_score: %d 0.717325


epochs:  129
prev_score: %d 0.7267
pres_score: %d 0.7142


epochs:  130
prev_score: %d 0.7267
pres_score: %d 0.726375


epochs:  131
prev_score: %d 0.7267
pres_score: %d 0.714025


epochs:  132
prev_score: %d 0.7267
pres_score: %d 0.716525


epochs:  133
prev_score: %d 0.7267
pres_score: %d 0.71225


epochs:  134
prev_score: %d 0.7267
pres_score: %d 0.722175


epochs:  135
prev_score: %d 0.7267
pres_score: %d 0.7175


epochs:  136
prev_score: %d 0.7267
pres_score: %d 0.71335


epochs:  137
prev_score: %d 0.7267
pres_score: %d 0.73065


epochs:  138
prev_score: %d 0.73065
pres_score: %d 0.721775


epochs:  139
prev_score: %d 0.7306

...

epochs:  179
prev_score: %d 0.747975
pres_score: %d 0.748675


epochs:  180
prev_score: %d 0.748675
pres_score: %d 0.7488


epochs:  181
prev_score: %d 0.7488
pres_score: %d 0.7491


epochs:  182
prev_score: %d 0.7491
pres_score: %d 0.74955


epochs:  183
prev_score: %d 0.74955
pres_score: %d 0.749975


epochs:  184
prev_score: %d 0.749975
pres_score: %d 0.7504


epochs:  185
prev_score: %d 0.7504
pres_score: %d 0.7507


epochs:  186
prev_score: %d 0.7507
pres_score: %d 0.7512


epochs:  187
prev_score: %d 0.7512
pres_score: %d 0.751675


epochs:  188
prev_score: %d 0.751675
pres_score: %d 0.752075


epochs:  189
prev_score: %d 0.752075
pres_score: %d 0.752425


epochs:  190
prev_score: %d 0.752425
pres_score: %d 0.75285


epochs:  191
prev_score: %d 0.75285
pres_score: %d 0.7532


epochs:  192
prev_score: %d 0.7532
pres_score: %d 0.753525


epochs:  193
prev_score: %d 0.753525
pres_score: %d 0.753875


epochs:  194
prev_score: %d 0.753875
pres_score: %d 0.754325


epochs:  195
prev_sc

epochs:  115
prev_score: %d 0.701775
pres_score: %d 0.702575


epochs:  116
prev_score: %d 0.702575
pres_score: %d 0.7033


epochs:  117
prev_score: %d 0.7033
pres_score: %d 0.704175


epochs:  118
prev_score: %d 0.704175
pres_score: %d 0.70505


epochs:  119
prev_score: %d 0.70505
pres_score: %d 0.7063


epochs:  120
prev_score: %d 0.7063
pres_score: %d 0.7069


epochs:  121
prev_score: %d 0.7069
pres_score: %d 0.7077


epochs:  122
prev_score: %d 0.7077
pres_score: %d 0.708425


epochs:  123
prev_score: %d 0.708425
pres_score: %d 0.709525


epochs:  124
prev_score: %d 0.709525
pres_score: %d 0.7103


epochs:  125
prev_score: %d 0.7103
pres_score: %d 0.710925


epochs:  126
prev_score: %d 0.710925
pres_score: %d 0.711625


epochs:  127
prev_score: %d 0.711625
pres_score: %d 0.71215


epochs:  128
prev_score: %d 0.71215
pres_score: %d 0.712925


epochs:  129
prev_score: %d 0.712925
pres_score: %d 0.713425


epochs:  130
prev_score: %d 0.713425
pres_score: %d 0.71435


epochs:  131
prev

epochs:  50
prev_score: %d 0.619225
pres_score: %d 0.622375


epochs:  51
prev_score: %d 0.622375
pres_score: %d 0.6254


epochs:  52
prev_score: %d 0.6254
pres_score: %d 0.62825


epochs:  53
prev_score: %d 0.62825
pres_score: %d 0.630875


epochs:  54
prev_score: %d 0.630875
pres_score: %d 0.6336


epochs:  55
prev_score: %d 0.6336
pres_score: %d 0.63615


epochs:  56
prev_score: %d 0.63615
pres_score: %d 0.63785


epochs:  57
prev_score: %d 0.63785
pres_score: %d 0.64015


epochs:  58
prev_score: %d 0.64015
pres_score: %d 0.64205


epochs:  59
prev_score: %d 0.64205
pres_score: %d 0.6441


epochs:  60
prev_score: %d 0.6441
pres_score: %d 0.646125


epochs:  61
prev_score: %d 0.646125
pres_score: %d 0.64815


epochs:  62
prev_score: %d 0.64815
pres_score: %d 0.649875


epochs:  63
prev_score: %d 0.649875
pres_score: %d 0.6515


epochs:  64
prev_score: %d 0.6515
pres_score: %d 0.653075


epochs:  65
prev_score: %d 0.653075
pres_score: %d 0.655025


epochs:  66
prev_score: %d 0.655025


epochs:  184
prev_score: %d 0.751175
pres_score: %d 0.751525


epochs:  185
prev_score: %d 0.751525
pres_score: %d 0.75185


epochs:  186
prev_score: %d 0.75185
pres_score: %d 0.75215


epochs:  187
prev_score: %d 0.75215
pres_score: %d 0.7525


epochs:  188
prev_score: %d 0.7525
pres_score: %d 0.75275


epochs:  189
prev_score: %d 0.75275
pres_score: %d 0.753125


epochs:  190
prev_score: %d 0.753125
pres_score: %d 0.753375


epochs:  191
prev_score: %d 0.753375
pres_score: %d 0.75375


epochs:  192
prev_score: %d 0.75375
pres_score: %d 0.754


epochs:  193
prev_score: %d 0.754
pres_score: %d 0.754425


epochs:  194
prev_score: %d 0.754425
pres_score: %d 0.7546


epochs:  195
prev_score: %d 0.7546
pres_score: %d 0.755


epochs:  196
prev_score: %d 0.755
pres_score: %d 0.7555


epochs:  197
prev_score: %d 0.7555
pres_score: %d 0.755825


epochs:  198
prev_score: %d 0.755825
pres_score: %d 0.756025


epochs:  199
prev_score: %d 0.756025
pres_score: %d 0.75655


epochs:  0
prev_score: %d

epochs:  97
prev_score: %d 0.7854
pres_score: %d 0.7864666666666666


epochs:  98
prev_score: %d 0.7864666666666666
pres_score: %d 0.7877166666666666


epochs:  99
prev_score: %d 0.7877166666666666
pres_score: %d 0.7886166666666666


epochs:  100
prev_score: %d 0.7886166666666666
pres_score: %d 0.7896


epochs:  101
prev_score: %d 0.7896
pres_score: %d 0.7905666666666666


epochs:  102
prev_score: %d 0.7905666666666666
pres_score: %d 0.7913666666666667


epochs:  103
prev_score: %d 0.7913666666666667
pres_score: %d 0.7923166666666667


epochs:  104
prev_score: %d 0.7923166666666667
pres_score: %d 0.7935


epochs:  105
prev_score: %d 0.7935
pres_score: %d 0.7940666666666667


epochs:  106
prev_score: %d 0.7940666666666667
pres_score: %d 0.7951833333333334


epochs:  107
prev_score: %d 0.7951833333333334
pres_score: %d 0.7962833333333333


epochs:  108
prev_score: %d 0.7962833333333333
pres_score: %d 0.7971333333333334


epochs:  109
prev_score: %d 0.7971333333333334
pres_score: %d 0.797

In [162]:
from sklearn.metrics import make_scorer, accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
scoring = {'f1 macro': make_scorer(f1_score , average='macro'), 'f1 micro': make_scorer(f1_score, average = 'micro'), 'Accuracy': make_scorer(accuracy_score) }

svm_grid = GridSearchCV(SVM(), params, cv=3, scoring=scoring, refit=False, n_jobs=-1)
svm_grid.fit(X, y)

svmcv_result = pd.DataFrame(svm_grid.cv_results_)
svmcv_result.to_csv("svmcv_result.csv")

In [167]:
class myClassifier(object):    
    """
    ovr
    """
    def __init__(self, C=1000, eta=0.1, batch_size=20, epochs=100, epsilon=1e-8, 
                 shuffle=True, params=None, w=0, b=0):
        self.C = C
        self.eta = eta
        self.batch_size = batch_size
        self.epochs = epochs
        self.epsilon = epsilon
        self.class_num = 0
        self.shuffle = shuffle
        self.update_count = 0
        self.w = 0
        self.b = 0
#         self.params['aver_w'] = w
#         self.params['aver_b'] = b
        
    def fit(self, X, y, params=None, w=0, b=0, testscore = None, eval_score=None):
        # X_num = m, X_fea = n
        # m = np.shape(X)[0], n = np.shape(X)[1]
        
        X_num, X_fea = np.shape(X)
        #X_num=60000 X_fea=28*28
        self.class_num=len(np.unique(y))
        #class_num=10
        
        if params is None:
            print('fit params=None')
            self.params = {
                'w': np.random.randn(X_fea, self.class_num), #(10, 784) 정규분포난수
                'b': np.random.randn(1, self.class_num),
                'w_': np.random.randn(X_fea, self.class_num),
                'b_': np.random.randn(1, self.class_num),
                'tmpw': 0,
                'tmpb': 0
            }
        cnt=1
        if eval_score is None:
            self.score_val = 0
                
        for Xi in range(self.epochs):
            s_data, s_labels = self.shuffling(X, y)
            encoded_y=self.encoding(s_labels)
            avg_loss = 0
            batch_count = int(X_num / self.batch_size)
            for t in range(int(batch_count)):
#               self.params['tmpw'] = temp_w, self.params['tmpb'] = temp_b
                batch_X, batch_y, bs=self.batching(s_data, encoded_y, t)
                batch_X = np.reshape(batch_X, (bs, X_fea))
                batch_y = np.reshape(batch_y, (bs, self.class_num))
                z = self.net_input(batch_X)
                loss = self.hinge_loss(batch_y, z)
                self.update_w_b(batch_X, batch_y, z, bs, cnt)
                cnt+=1
                avg_loss += loss
                self.update_count += 1
##ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
# aver_w = w_ , w_ = w
            self.params['tmpw'] = (cnt * (cnt/(cnt+1)) * 
                                   self.params['w_'] + (1/(cnt+1))*self.params['w'])
            self.params['tmpb'] = (cnt * (cnt/(cnt+1)) * 
                                   self.params['b_'] + (1/(cnt+1))*self.params['b'])
            prev_score = self.score_val
            pres_score = self.score(X, y)
            print("epochs: ", Xi)
            print("prev_score: %d", prev_score)
            print("pres_score: %d\n", pres_score)
            if prev_score < pres_score:
                self.score_val = pres_score
            if self.det_weight(X, y, 1) < self.det_weight(X, y): # temp_w, temp_b
                self.params['w_'] = self.params['tmpw']
                self.params['b_'] = self.params['tmpb']
            avg_loss /= batch_count
        return self
    
    def det_weight(self, X, y, aver=0):
        if aver:
            w1 = self.params['w_']
            b1 = self.params['b_']
        else:
            w1 = self.params['tmpw']
            b1 = self.params['tmpb']
        temp = np.dot(X, w1) + b1
#         temp = temp.T
        pred = np.argmax(temp, axis=1)
        sco = np.mean(pred == y)
        return sco
    
    def update_w_b(self, batch_X, batch_y, z, bs, cnt):
        n = np.shape(batch_X)[1]  # num of features
        delta_w = np.zeros(self.params['w'].shape)
        delta_b = np.zeros(self.params['b'].shape)
        z = np.reshape(z, (bs, self.class_num))
        temp = 1 - np.multiply(batch_y, z)
        temp[temp <= 0] = 0
        temp[temp > 0] = 1
        y_temp = np.multiply(batch_y, temp.reshape(bs, self.class_num))
        delta_w = -(1 / bs) * np.matmul(batch_X.T, y_temp) + (1 / self.C) * self.params['w']
        delta_b = -(1 / bs) * np.sum(y_temp, axis=0)
        self.params['w'] = self.params['w'] - (self.eta / (1 + self.epsilon * cnt)) * delta_w
        self.params['b'] = self.params['b'] - (self.eta / (1 + self.epsilon * cnt)) * delta_b
        
        return self.params
    
    def hinge_loss(self, y, z):
        loss = 1 - np.multiply(y, z)
        loss[loss < 0] = 0
        loss = np.mean(loss)
        return loss
    
    def net_input(self, X):  # net_input() = forward_prop(), generate z
        z = np.matmul(X, self.params['w']) + self.params['b']
        return z

    def encoding(self, y):
        encoded_y=-1*np.ones((np.shape(y)[0],self.class_num))
        for i in range(np.shape(y)[0]):
            encoded_y[i,y[i]] = 1
        return encoded_y
                
    def shuffling(self, X, y):
        temp_s=list(zip(X,y))
        random.shuffle(temp_s)
        X,y=zip(*temp_s)
        return X,y
    
    def batching(self, X, y, t):                         
        batch_X = X[t * self.batch_size : min(len(X), (t+1) * self.batch_size)]
        batch_y = y[t * self.batch_size : min(len(X), (t+1) * self.batch_size)]
        last_size = min(len(X), (t+1) * self.batch_size) - t * self.batch_size
        
        return batch_X, batch_y,last_size
    
    def predict(self, X):
        m = np.shape(X)[0]
        class_score = self.net_input(X)  # return z
        pred = np.argmax(class_score, axis=1)

        return pred
    
    def score(self, X, y):
        pred = self.predict(X)
        score = np.mean(pred == y)
        
        return score
    
    def get_params(self, deep=True):
        return {'C':self.C, 'batch_size':self.batch_size, 'epochs':self.epochs,
               'eta': self.eta}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

In [168]:
import random
SupportVM = myClassifier().fit(X, y)

fit params=None
epochs:  0
prev_score: %d 0
pres_score: %d
 0.7989666666666667
epochs:  1
prev_score: %d 0.7989666666666667
pres_score: %d
 0.8327833333333333
epochs:  2
prev_score: %d 0.8327833333333333
pres_score: %d
 0.8419333333333333
epochs:  3
prev_score: %d 0.8419333333333333
pres_score: %d
 0.83825
epochs:  4
prev_score: %d 0.8419333333333333
pres_score: %d
 0.8405166666666667
epochs:  5
prev_score: %d 0.8419333333333333
pres_score: %d
 0.8357333333333333
epochs:  6
prev_score: %d 0.8419333333333333
pres_score: %d
 0.8436333333333333
epochs:  7
prev_score: %d 0.8436333333333333
pres_score: %d
 0.8488833333333333
epochs:  8
prev_score: %d 0.8488833333333333
pres_score: %d
 0.8472333333333333
epochs:  9
prev_score: %d 0.8488833333333333
pres_score: %d
 0.8507833333333333
epochs:  10
prev_score: %d 0.8507833333333333
pres_score: %d
 0.8308166666666666
epochs:  11
prev_score: %d 0.8507833333333333
pres_score: %d
 0.8496833333333333
epochs:  12
prev_score: %d 0.8507833333333333
pres